# Module 5 Homework - Let's Get Lunch

Suppose we are looking for a good place for lunch on our next trip to La Crosse. Let's collect some data on nearby restaurants.

## <font color="red"> Problem 1 </font>

Goto yelp.com and perform a search with the following parameters

* **Find** Restaurants
* **Near** La Crosse, WI

**Tasks**
1. Copy the resulting web address below and determine the how the     specified search terms related to the resulting address
2. Use requests and Beautiful Soap to download the content of the front page.

In [2]:
# Import modules here
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
from composable import pipeable
from composable import from_toolz as tlz
from composablesoup.soup import find_parent, parents, children, find_previous_sibling, find_previous_siblings, find_next_sibling, find_next_siblings, find_previous_sibling
import requests
import re as r
from bs4 import BeautifulSoup

In [3]:
# Get and process the Yelp search
s = requests.Session()
r = s.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI')
lunch = BeautifulSoup(r.content, 'html.parser')

# in URL, find_desc= refers to description (I entered Restaurants), 
# and find_loc= refers to near what location (I entered La Crosse, WI). 
# It seems that %20 is a 'space' character, and %2C is a ',' character

## <font color="red"> Problem 2 </font>

We want to grab the restaurant's name.

1. Use Inspect Element to determine the tags/classes for each of the elements listed above.  
2. Note that all the business information is contained in a`div` that contains a class similar to  `<div class=" ... businessName__09f24__3Wql2 ...">`.  You will need to use a regular expression to match the `businessName` in the class (see lecture 5.3).
3. Write expressions/functions to pull out the name of each restaurant.  
    * Note: The business name is in an unnamed tag, you will need to navigate to the using searches and/or relationship
 
**Confirm that there is an extra restaurant in the list (e.g. 11-12 instead of 10). This is due to an advertisement/sponsered links, which we want to ignore.**

In [4]:
(
    lunch
    >> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
    >> map(find('a'))
    >> map(get_text)
)

# Pappi's Taqueria y Mas is extra

['Pappi’s Taqueria y Mas',
 'The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

In [6]:
import re as r
rest_name = r.compile(r'(\d+\.)(.*)')
check_rest_name = pipeable(lambda s: rest_name.match(s) if rest_name.match(s) else None)
just_name = pipeable(lambda s: s.split()[0].strip() if check_rest_name(s) != None else None)

get_restaurant_name = pipeable(lambda soup: 
soup
    >> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
    >> map(get_text)
    >> map(just_name)
)
# this leaves the first one which must be some sort of advertisement

In [7]:
lunch >> get_restaurant_name

[None, '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.']

In [8]:
def get_rest_name(soup):
    """
    """
    lis = (soup
    >> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
    >> map(get_text)
    >> map(just_name)
    )
    names = [s for s in lis if type(s) != None]

    return names

In [9]:
get_rest_name(lunch)

[None, '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.']

In [10]:
fix_name = pipeable(lambda s: s.replace('\xa0', ' '))
list = (lunch
    >> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
    >> map(get_text)
)
names = (list[1:]
    >> map(fix_name)
)
names

['1. The Waterfront Restaurant & Tavern',
 '2. Restore Public House',
 '3. Buzzard Billy’s',
 '4. Lovechild Restaurant',
 '5. Howie’s on La Crosse',
 '6. The Charmant',
 '7. The Freighthouse Restaurant',
 '8. The Crow',
 '9. Schuby’s Neighborhood Butcher',
 '10. Piggy’s Restaurant']

## <font color="red"> Problem 3 </font>

Since we picked up extra information, we will need to be clever about identifying the information block for each restaurant.  Note that all of the actual search results (but not sponsered links) start with the ranking (e.g. `11.`).  Use the following steps to get a list that contains the information for each restaurant other than the adds.

1. Start by finding the ranking of the restaurant (1., 2., etc.). **Hint:** You will need to use regular expression to match the text of the tag (see lecture 5.3).
2. Now search for a parent of the above tags that surrounds all of the restaurant information.  You will want to use the `find_parent` method on each of the tags from **1.**.  **Hint:** Look through each of the `div` tags that contain the ranking, looking for a meaningful tag name to match with a regular expression.

The resulting list will be the starting point for gathering all of the information.

In [11]:
drop_first = pipeable(lambda s: tlz.drop(1, s))
start = (lunch
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> drop_first
)

In [12]:
start >> map(get_text)

['1.\xa0The Waterfront Restaurant & Tavern226$$$American (New), Seafood, Steakhouses(608) 782-5400328 Front St S',
 '2.\xa0Restore Public House17American (Traditional)(608) 782-52101810 State St',
 '3.\xa0Buzzard Billy’s277$$American (Traditional), Cajun/Creole(608) 796-2277222 Pearl St',
 '4.\xa0Lovechild Restaurant109$$$American (New)(608) 433-2234300 3rd St S',
 '5.\xa0Howie’s on La Crosse57$$American (New), Pubs(608) 784-74001128 La Crosse St',
 '6.\xa0The Charmant149$$French, Cocktail Bars(608) 519-8830101 State St',
 '7.\xa0The Freighthouse Restaurant125$$$Seafood, Steakhouses, Desserts(608) 784-6211107 Vine St',
 '8.\xa0The Crow154$$American (Traditional), Gastropubs(608) 519-5400100 3rd St S',
 '9.\xa0Schuby’s Neighborhood Butcher17$$Butcher, Delis, Caterers(608) 615-1076321 State St',
 '10.\xa0Piggy’s Restaurant131$$Steakhouses, Seafood, Sandwiches(608) 784-4877501 Front St S']

## <font color="red"> Problem 4 </font>

Write expressions/functions to gather each of the following pieces of information for each of the restaurants.

#### Restaurant Name

In [13]:
get_restaurant_name = pipeable(lambda soup:(soup
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> map(find('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'}))
>> map(find('a'))
>> drop_first
>> map(get_text)
))

In [14]:
lunch >> get_restaurant_name

['The Waterfront Restaurant & Tavern',
 'Restore Public House',
 'Buzzard Billy’s',
 'Lovechild Restaurant',
 'Howie’s on La Crosse',
 'The Charmant',
 'The Freighthouse Restaurant',
 'The Crow',
 'Schuby’s Neighborhood Butcher',
 'Piggy’s Restaurant']

#### Rating

In [15]:
rating_regex = r.compile(r'\d\.?\d?\sstar\srating')
get_stars = pipeable(lambda s: find(soup=s, name='div', attrs={'aria-label':rating_regex})['aria-label'] if (s >> find('div', attrs={'aria-label':rating_regex})) != None else 'No rating')

get_rating = pipeable(lambda soup:(soup
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> drop_first
>> map(get_stars)
))

In [16]:
lunch >> get_rating

['4.5 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4 star rating',
 '4.5 star rating',
 '4 star rating',
 '3.5 star rating',
 '4.5 star rating',
 '4 star rating']

#### Address

In [17]:
address_missing = pipeable(lambda s: (s >> find('address') >> get_text) if (s >> find('address')) != None else '')


get_address = pipeable(lambda soup:(soup
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> drop_first
>> map(address_missing)
))
#<div class=" secondaryAttributes__09f24__3db5x arrange-unit__09f24__1gZC1 border-color--default__09f24__R1nRO"><div class=" container__09f24__1fWZl padding-l2__09f24__2MHQ3 border-color--default__09f24__R1nRO text-align--right__09f24__wJEQr"><div class=" border-color--default__09f24__R1nRO"><div class=" display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO"><div class=" border-color--default__09f24__R1nRO"><p class=" text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--right__09f24__1TIxB text-size--small__09f24__1Z_UI">(608) 782-5400</p></div></div></div><address class=""><div class=" border-color--default__09f24__R1nRO"><div class=" display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO"><div class=" border-color--default__09f24__R1nRO"><p class=" text__09f24__2tZKC text-color--black-extra-light__09f24__38DtK text-align--right__09f24__1TIxB text-size--small__09f24__1Z_UI"><span class=" raw__09f24__3Obuy">328 Front St S</span></p></div></div></div></address><div class=" margin-b1__09f24__1647o border-color--default__09f24__R1nRO"></div></div></div>

In [18]:
lunch >> get_address

['328 Front St S',
 '1810 State St',
 '222 Pearl St',
 '300 3rd St S',
 '1128 La Crosse St',
 '101 State St',
 '107 Vine St',
 '100 3rd St S',
 '321 State St',
 '501 Front St S']

#### Review Count

In [19]:
num_reviews_missing = pipeable(lambda s: (s >> find('span', attrs={'class':'text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0'}) >> get_text) if (s >> find('span', attrs={'class':'text__09f24__2tZKC reviewCount__09f24__EUXPN text-color--black-extra-light__09f24__38DtK text-align--left__09f24__3Drs0'})) != None else '0')

get_num_reviews = pipeable(lambda soup:(soup
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> drop_first
>> map(num_reviews_missing)
))

In [20]:
lunch >> get_num_reviews

['226', '17', '277', '109', '57', '149', '125', '154', '17', '131']

#### Category

**Examples:** `['American (New)', 'Seafood', 'Steakhouses']` becomes `'American (New);Seafood;Steakhouses'`

In [21]:
combine_categories = pipeable(lambda s: ';'.join(s))

get_category = pipeable(lambda soup:(soup
>> find_all('div', attrs={'class':'businessName__09f24__3Wql2 display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'})
>> map(find_parent(name='div', attrs={'class':'scrollablePhotos__09f24__1PpB8 arrange__09f24__AiSIM border-color--default__09f24__R1nRO'}))
>> drop_first
>> map(find('div', attrs={'class':'priceCategory__09f24__2IbAM display--inline-block__09f24__FsgS4 border-color--default__09f24__R1nRO'}))
>> map(find_all('a'))
>> map(map(get_text))
>> map(combine_categories)
))

In [22]:
lunch >> get_category

['American (New);Seafood;Steakhouses',
 'American (Traditional)',
 'American (Traditional);Cajun/Creole',
 'American (New)',
 'American (New);Pubs',
 'French;Cocktail Bars',
 'Seafood;Steakhouses;Desserts',
 'American (Traditional);Gastropubs',
 'Butcher;Delis;Caterers',
 'Steakhouses;Seafood;Sandwiches']

## <font color="red">  Problem 4 </font>

Package all of the expressions in a function that takes a url as input and returns the table of information.  Use a `def` statement and put the above helper functions in the body of the main function.  Test this function on the front page of the search.

In [23]:
def get_all_restaurant_info(site):
    """ 
    ARGS: a URL of yelp 
    returns: list of lists of all restaurant data on page
    """
    s = requests.Session()
    r = s.get(site)
    soup = BeautifulSoup(r.content, 'html.parser')

    restaurant_name = get_restaurant_name(soup)
    rating = get_rating(soup)
    num_reviews = get_num_reviews(soup)
    address = get_address(soup)
    category = get_category(soup)
    output = [row for row in zip(restaurant_name, rating, num_reviews, address, category)]
    return output

In [24]:
get_all_restaurant_info('https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse,%20WI')

[('The Waterfront Restaurant & Tavern',
  '4.5 star rating',
  '226',
  '328 Front St S',
  'American (New);Seafood;Steakhouses'),
 ('Restore Public House',
  '4.5 star rating',
  '17',
  '1810 State St',
  'American (Traditional)'),
 ('Buzzard Billy’s',
  '4 star rating',
  '277',
  '222 Pearl St',
  'American (Traditional);Cajun/Creole'),
 ('Lovechild Restaurant',
  '4.5 star rating',
  '109',
  '300 3rd St S',
  'American (New)'),
 ('Howie’s on La Crosse',
  '4 star rating',
  '57',
  '1128 La Crosse St',
  'American (New);Pubs'),
 ('The Charmant',
  '4.5 star rating',
  '149',
  '101 State St',
  'French;Cocktail Bars'),
 ('The Freighthouse Restaurant',
  '4 star rating',
  '125',
  '107 Vine St',
  'Seafood;Steakhouses;Desserts'),
 ('The Crow',
  '3.5 star rating',
  '154',
  '100 3rd St S',
  'American (Traditional);Gastropubs'),
 ('Schuby’s Neighborhood Butcher',
  '4.5 star rating',
  '17',
  '321 State St',
  'Butcher;Delis;Caterers'),
 ('Piggy’s Restaurant',
  '4 star rating'

## <font color="red">  Problem 5 </font>

Now perform a linear search to grab all of the information on restaurants in La Crosse.  You will need to

1. Inspect the url for the first, second, etc. pages to determine the pattern.
2. Create a list of urls for all search results.
3. Get the info from all pages and use your function from the last problem to parse the data into a list.
4. Write the results to a csv file. **Hint:** Use `'a'` (append) instead of `'w'` write on all after the first/

In [25]:
base = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start={start}'
yelps = [base.format(start = h) for h in ['0','10','20','30','40','50','60','70','80','90','100','110','120','130','140','150','160','170','180','190','200','210','220','230']]
yelps


['https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=0',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=10',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=20',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=30',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=40',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=50',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=60',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=70',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=80',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&start=90',
 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=La%20Crosse%2C%20WI&

In [29]:
restaurants_list = [get_all_restaurant_info(l) for l in yelps]

In [27]:
each_restaurant = [row for restaurants_list in restaurants_list for row in restaurants_list]
contents = [','.join(row) for row in each_restaurant]
contents[:10]

['The Waterfront Restaurant & Tavern,4.5 star rating,226,328 Front St S,American (New);Seafood;Steakhouses',
 'Restore Public House,4.5 star rating,17,1810 State St,American (Traditional)',
 'Buzzard Billy’s,4 star rating,277,222 Pearl St,American (Traditional);Cajun/Creole',
 'Lovechild Restaurant,4.5 star rating,109,300 3rd St S,American (New)',
 'Howie’s on La Crosse,4 star rating,57,1128 La Crosse St,American (New);Pubs',
 'The Charmant,4.5 star rating,149,101 State St,French;Cocktail Bars',
 'The Freighthouse Restaurant,4 star rating,125,107 Vine St,Seafood;Steakhouses;Desserts',
 'The Crow,3.5 star rating,154,100 3rd St S,American (Traditional);Gastropubs',
 'Schuby’s Neighborhood Butcher,4.5 star rating,17,321 State St,Butcher;Delis;Caterers',
 'Piggy’s Restaurant,4 star rating,131,501 Front St S,Steakhouses;Seafood;Sandwiches']

In [498]:
with open('La_Crosse_Restaurants.csv', 'w') as outfile:
    header = 'Name,Rating,Number Of Reviews,Address,Category'
    print(header, file=outfile)
    count = 0
    for yelp in yelps:
        for line in contents:
            print(line, file=outfile)

## <font color="red">  Bonus Problem </font>

See if you can also grab the latitude and longitude of each result.